In [1]:
from modules.data_visualizer import ImageVisualizer
from modules.data_loader import DataLoaderCreator
from modules.data_prediction import ComprehensiveImagePredictor
from modules.model_eval import ModelEvaluator
from modules.model_performance import ModelPerformanceVisualizer
from modules.model_trainer import ModelTrainer
from modules.model_saver import ModelSaver

In [ ]:
SPLITTED_DATASET = "2_CLASS_DS"
train_dir = f"{SPLITTED_DATASET}/train"
test_dir  = f"{SPLITTED_DATASET}/test"

train_dir,test_dir

visualizer = ImageVisualizer(SPLITTED_DATASET)
visualizer.visualize_random_image()

In [3]:
import torch
from torchvision import models
from torchvision import transforms
from torchvision.transforms import InterpolationMode
from torch import nn

weights = models.MobileNet_V2_Weights.DEFAULT
mnet_transform = weights.transforms()
print(mnet_transform)

print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"Device name: {torch.cuda.get_device_name(0)}")
    print(f"Device count: {torch.cuda.device_count()}")
else:
    print("CUDA is not available. Using CPU.")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
ASL_mobilenet = models.mobilenet_v2(weights=weights).to(device)

ImageClassification(
    crop_size=[224]
    resize_size=[232]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BILINEAR
)
CUDA available: True
CUDA version: 12.1
Device name: NVIDIA GeForce GTX 1070
Device count: 1


In [ ]:
train_transforms = mnet_transform

# test_transforms = mnet_transform

visualizer.plot_transformed_images(train_transforms,n=3, seed=None)

In [ ]:
from torch.optim.lr_scheduler import ReduceLROnPlateau

# Freeze Last Layer
for param in ASL_mobilenet.features.parameters():
    param.requires_grad = False

loader_creator = DataLoaderCreator(train_dir=train_dir,
                                   test_dir=test_dir,
                                   transform=train_transforms)

train_dataloader, test_dataloader, class_names = loader_creator.create_dataloaders(batch_size=32)

class_names_count = len(loader_creator.get_class_names())

print(f"Class names: {loader_creator.get_class_names()}")
print(f"Class names Count: {class_names_count}")

trainer = ModelTrainer()

NUM_EPOCHS = 100
EARLY_STOPPING = 1
LEARNING_RATE = 0.001

ASL_mobilenet.classifier = torch.nn.Sequential(
    torch.nn.Dropout(p=0.2, inplace=True), 
    torch.nn.Linear(in_features=1280, 
                    out_features=class_names_count,
                    bias=True)).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=ASL_mobilenet.parameters(), lr=LEARNING_RATE)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=2)

TRAINED_MODEL, RESULTS = trainer.train(
    model=ASL_mobilenet,
    train_loader=train_dataloader,
    test_loader=test_dataloader,
    optimizer=optimizer,
    loss_fn=criterion,
    epochs=NUM_EPOCHS,
    scheduler=scheduler,
    patience=EARLY_STOPPING 
)

visualizer = ModelPerformanceVisualizer(RESULTS)
y_true, y_pred = visualizer.get_preds(model=TRAINED_MODEL, dataloader=test_dataloader, device=device)
visualizer.plot_all(y_true=y_true, y_pred=y_pred, classes=class_names, save_path="model_performance/plot.jpg")

In [ ]:
saver = ModelSaver(target_dir="models")
saver.save_model(model=ASL_mobilenet, model_name="2_CLASSES_MODEL.pth")
loaded_model = saver.load_model(model=ASL_mobilenet, model_name="2_CLASSES_MODEL.pth")
# loaded_model.state_dict()


In [ ]:
predictor = ComprehensiveImagePredictor(loaded_model, class_names, train_transforms)
predictor.predict_and_visualize_multiple(model=loaded_model, directory=test_dir, transformer=train_transforms, num_samples=9, seed=None)

file_path = "./2_CLASS_DS/test/1"

In [14]:
evaluator = ModelEvaluator(loaded_model)
results = evaluator.evaluate_and_report(test_dataloader, class_names)

print(f"Accuracy: {results['accuracy']:.4f}")
print("Classification Report:")
print(results['classification_report'])

Accuracy: 1.0000
Classification Report:
              precision    recall  f1-score   support

           1       1.00      1.00      1.00        20
           A       1.00      1.00      1.00        20

    accuracy                           1.00        40
   macro avg       1.00      1.00      1.00        40
weighted avg       1.00      1.00      1.00        40



#### Prep model for Android

#### Save for android

In [18]:
import torch
import torchvision.models as models
import os

OUT_FEATURES = 2
MODEL_NAME = "2_CLASSES_MODEL.pth"
MODEL_PATH = os.path.join("models", MODEL_NAME)
PATH_TO_SAVE_MODEL = os.path.join("models", "android", MODEL_NAME.replace(".pth", ""))

# Ensure the directory exists
os.makedirs(os.path.dirname(PATH_TO_SAVE_MODEL), exist_ok=True)

model = models.mobilenet_v2(pretrained=False)
model.classifier[1] = torch.nn.Linear(in_features=1280, out_features=OUT_FEATURES)

# Load the state dict from the file
model.load_state_dict(torch.load(MODEL_PATH))

model.eval()

example_input = torch.rand(1, 3, 232, 232)  # Batch size 1, 3 channels, 232x232 pixels
traced_model = torch.jit.trace(model, example_input)

# Save the traced model
traced_model._save_for_lite_interpreter(f"{PATH_TO_SAVE_MODEL}.ptl")
print(f"Model saved to {PATH_TO_SAVE_MODEL}.ptl")

Model saved to models\android\2_CLASSES_MODEL.ptl


In [20]:
import torch

lite_model = torch.jit.load(f"./models/android/2_CLASSES_MODEL.ptl")

example_input = torch.rand(1, 3, 224, 224)
output = lite_model(example_input)

print(f"Output shape: {output.shape}")

Output shape: torch.Size([1, 2])
